# Data cleaning

## Goal

In this notebook, we will be taking in the raw csv data and cleaning / parsing its different columns. The notebook contains the transformations below in order:

1. Read in the data
2. Removing unused columns for this analysis
3. Removing rows with certain null columns
4. Cleaning of columns
    * ad_age
    * ad_impressions
    * ad_clicks
    * ad_creation_date
    * ad_end_date
    * ad_targeting_interests
    * ad_targeting_people_who_match
5. Writing to file
6. Summary of lost rows


We first import 3 packages which will be useful for our data cleaning:

* pandas for handling csv as a table
* numpy to handle mulitdimensional array operations
* re to handle regular expression parsing

In [2]:
import pandas as pd
import numpy as np
import re

# We read in the data
ads_df = pd.read_csv('../raw_data/raw.csv')

# Output first 2 rows
ads_df.head(2)

,Unnamed: 0,file_name,ad_id,ad_text,ad_landing_page,ad_targeting_custom_audience,ad_targeting_location,ad_targeting_interests,ad_targeting_excluded_connections,ad_targeting_age,ad_targeting_language,ad_targeting_placements,ad_targeting_people_who_match,ad_impressions,ad_clicks,ad_spend,ad_creation_date,ad_end_date
0,0,P(1)0003283.txt,1349,Close the borders until it's too late.\nLike t...,https://www.facebook.com/StopAllInvaders,NaN,United States,NaN,Exclude people who like Stop A.1.,35 - 65+,English (UK)or English (US),News Feed on desktop computers or News Feed on...,Interests: Immigration or Politics,1,0,0.66 RUB,04/13/16 07:48:33 AM PDT,NaN
1,1,P(1)0002823.txt,986,repost @[1029022833877757:274:Blackslayingit]\n,https://www.facebook.com/blacktivists/,NaN,United States,NaN,NaN,16 - 65+,English (UK) or English (US),News Feed on desktop computers or News Feed on...,"Interests: Pan-Africanism, African-American Ci...",10.496,1.823,200.00 RUB,04/21/17 07:42:45 AM PDT,04/22/17 11:00:00 AM PDT


## Removing unused columns

Our first step will be to remove columns we will not be using for this analysis.

| Column name | Reason for removal|
|-------------| ------------------|
| Unnamed | Index column added by accident in the data production step. |
| ad_id | We will be using the file_name column as identifier. |
| ad_text | Although interesting its analysis is outside the scope of this project | 
| ad_landing_page | This will not be useful in answering our research questions. |
| ad_targeting_location | We will not be studying the impact of location for the ads |
| ad_targeting_custom_audience | This field doesn't contain information not already in the ad_targeting_interests column |
| ad_targeting_language | This field is almost always US English and is not present on most of the dataset |
| ad_targeting_placements | We will not be studying the impact of location on the page for the ads |

In [3]:
# Columns we will not be using
columns_to_remove = ['Unnamed: 0', 'ad_id', 'ad_text', 'ad_landing_page', 'ad_targeting_location', 'ad_targeting_custom_audience', 'ad_targeting_excluded_connections', 'ad_targeting_language', 'ad_targeting_placements']
ads_df = ads_df.drop(columns=columns_to_remove)

ads_df.head(2)

,file_name,ad_targeting_interests,ad_targeting_age,ad_targeting_people_who_match,ad_impressions,ad_clicks,ad_spend,ad_creation_date,ad_end_date
0,P(1)0003283.txt,NaN,35 - 65+,Interests: Immigration or Politics,1,0,0.66 RUB,04/13/16 07:48:33 AM PDT,NaN
1,P(1)0002823.txt,NaN,16 - 65+,"Interests: Pan-Africanism, African-American Ci...",10.496,1.823,200.00 RUB,04/21/17 07:42:45 AM PDT,04/22/17 11:00:00 AM PDT


## Removing rows with null columns

We will be removing rows with null for: ad_creation_date, ad_spend, ad_targeting_age, ad_impressions and ad_clicks. Our first step will be to create a dictionary which can keep track of the number of rows remaining after a given operiation. We will be using this dictionary when summarizing the cleaning of the dataset and its repercussions on our analysis. We then create a function which remove the null values for a column name and adds the row count after removal to our summary dictionary.

In [4]:
# Dictionary to keep track of row removals
cleaning_summary_format = {'before_cleaning_count': len(ads_df)}

# Function to remove null rows for a given column
def remove_nulls(ads_df, column_name):
    # np.where returns a tuple, we want the first member (the indexes of rows)
    null_indexes = np.where(pd.isnull(ads_df[column_name]))[0]
    # We drop the list of indexes
    ads_df = ads_df.drop(null_indexes)
    # We add the entry to our summary
    cleaning_summary_format['null_' + column_name + '_count'] = len(ads_df)
    return ads_df

# We remove nulls for given columns
ads_df = remove_nulls(ads_df, 'ad_creation_date')
ads_df = remove_nulls(ads_df, 'ad_spend')
ads_df = remove_nulls(ads_df, 'ad_targeting_age')
ads_df = remove_nulls(ads_df, 'ad_impressions')
ads_df = remove_nulls(ads_df, 'ad_clicks')

In [5]:
print('''Before cleaning our dataset had {before_cleaning_count} columns.
After removing rows with null creation dates: {null_ad_creation_date_count} columns.
After removing rows with null ad spending: {null_ad_spend_count} columns.
After removing rows with null ad targeting age: {null_ad_targeting_age_count} columns.
After removing rows with null ad impressions: {null_ad_impressions_count} columns.
After removing rows with null ad clicks: {null_ad_clicks_count} columns.'''.format(**cleaning_summary_format))

Before cleaning our dataset had 3517 columns.
After removing rows with null creation dates: 3497 columns.
After removing rows with null ad spending: 3497 columns.
After removing rows with null ad targeting age: 3497 columns.
After removing rows with null ad impressions: 3497 columns.
After removing rows with null ad clicks: 3497 columns.


### Cleaning ad_age

First we look at the values for the field and whether we will be able to leverage them in our analysis.

In [6]:
ads_df.ad_targeting_age.value_counts().index

Index(['18 - 65+', '16 - 65+', '18 - 54', '18 - 45', '18 - 65+ Gender: Male',
       '18 - 51', '14 - 40', '13 - 65+', '18 - 50', '21 - 65+', '35 - 65+',
       '14 - 65+', '13 - 30 Gender: Male', '15 - 25', '15 - 25 Gender: Female',
       '18 - 59', '19 - 30', '16 - 45', '15 - 30', '13 - 45', '13 - 40',
       '16 - 53', '14 - 54', '25 - 65+', '13 - 35', '14 - 40 Gender: Female',
       '14 - 17 Gender: Female', '45 - 64 Gender: Male', '16 - 25',
       '17 - 65+ Gender: Male', '18 - 60', '25 - 65+ Gender: Male', '24 - 45',
       '16 - 40', '18 65+', '15 - 40', '15 - 54', '18 - 39', '30 - 65+',
       '18 - 65+ Gender: Female', '16 - 55', '45 - 65+', '18 - 48', '13 - 44',
       '20 - 45', '13 - 50', '20 - 65+', '13 - 60', '18 - 55', '24 - 65+',
       '16 - 60', '15 - 65+', '18 - 61', '18 - 24', '15 - 26', '18- 51',
       '16 - 54', '13 - 55', '15 - 25 Gender: Male', '16 - 50', '14 - 50',
       '18 - 43', '13 - 42', '18 - 53', '20 - 35', '40 - 65+', '13 - 27',
       '14 - 36', '

The initial parsing for this field was not perfect... Let's simplify this bucketing by removing gender information. To do so let's crop the string at 8 characters.

In [7]:
# Crop the ad_targeting_age to 8 characters
ads_df.ad_targeting_age = ads_df.ad_targeting_age.apply(lambda s: s if len(s)<=8 else s[0:8])

# Count rows for the different values
count_table = ads_df.ad_targeting_age.value_counts().to_frame()

# Rename the column for clarity
count_table.columns = ['Ad count']

# Output top 5
count_table.head(5)

,Ad count
18 - 65+,2347
16 - 65+,356
18 - 54,123
18 - 45,71
18 - 51,48


As per this table, almost all ads targeted voting age facebook users (18+). Bucketing the ads by age groups will not result in significant/interesting analysis. We drop the column.

In [8]:
ads_df = ads_df.drop(columns=['ad_targeting_age'])

### Cleaning ad_impressions and ad_clicks

Both these columns are numerical and do not contain None and or NaN values. The Oxford and the multiple university studies, mention that ads without impressions or clicks where unlikely to have been shown to Facebook users.

* We will first be parsing these fields that sometimes use . or , to separate thousands.
* We will then remove 0 values.

The studies mention that this removed quite a few entries.

In [9]:
# Parsing of string to integer
def format_string_to_integer(string):
    # Removing dots and commas and semicolons
    s = string.replace(',', '').replace('.', '').replace(';', '')
    
    # Removing typos betwee o, O (lower, upper letter o) and 0 (zero digit)
    s = s.replace('o', '0').replace('O', '0')
    
    # Accidental whitespace
    s = s.replace(' ', '')
    
    # Coerce string to integer
    return int(s)

# Remvoe rows with 0s and adds count after removal to summary
def remove_zeros(ads_df, column_name):
    ads_df = ads_df[ads_df[column_name] != 0]
    cleaning_summary_format['zeros_' + column_name + '_count'] = len(ads_df)
    return ads_df

In [10]:
# How many columns do we have before removing zeros
cleaning_summary_format['before_zeros_count'] = len(ads_df)

# Conversion to integers
ads_df['ad_clicks'] = ads_df['ad_clicks'].apply(format_string_to_integer)
ads_df['ad_impressions'] = ads_df['ad_impressions'].apply(format_string_to_integer)

# Removing zeros values
ads_df = remove_zeros(ads_df, 'ad_impressions')
ads_df = remove_zeros(ads_df, 'ad_clicks')

# Reporting
print('''Before removing 0 ad_impressions or ad_clicks our dataset had {before_zeros_count} columns.
After removing rows with 0 ad impressions: {zeros_ad_impressions_count} columns.
After removing rows with 0 ad clicks: {zeros_ad_clicks_count} columns.'''.format(**cleaning_summary_format))

Before removing 0 ad_impressions or ad_clicks our dataset had 3497 columns.
After removing rows with 0 ad impressions: 2588 columns.
After removing rows with 0 ad clicks: 2450 columns.


### Parsing creation date and end date

Creation date and end date are written in a complex format: 04/13/16 07:48:33 AM PDT. Our analysis only requires a subset of this information, the date. In this section we will extract the first 8 characters mm/dd/yy and convert them to a datetime object. Let's take a look at the entries:

In [11]:
ads_df['ad_creation_date']

1        04/21/17 07:42:45 AM PDT
2        04/13/17 04:40:03 AM PDT
5        05/29/17 12:29:07 AM PDT
6       08/18/16 08:31 :16 AM PDT
7        05/21/17 07:27:26 AM PDT
                  ...            
3511      04/13/17 0&14:34 AM PDT
3512    11 /18/16 08:22:22 AM PST
3513     01/24/17 06:08:31 AM PST
3515     02/07/17 04:30:54 AM PST
3516     02/22/17 06:49:48 AM PST
Name: ad_creation_date, Length: 2450, dtype: object

We find that sometimes the first few characters contain spaces. We write a regular expression for this and apply the removal of these white space part of a function. We also need to complete the year to be 4 characters for later date parsing.

In [12]:
# We first compile our date extraction regex to improve performance
date_regex = re.compile(r'(?P<date>\d\s*\d\s*\/\s*\d\s*\d\s*\/\s*\d\s*\d)')

# Given a string beginning with mm/dd/yy we produce mm/dd/YYYY
# Function returns 'parse_error' on failure to parse and null if the
# input string was null
def extract_date_from_string(string):
    matches = None
    date = None
    
    # If the string is not null attempt to find matches
    if not pd.isnull(string):
        matches = date_regex.search(string)
    else:
        # null value for string in pandas
        date = np.nan    

    # If the ?P<date> group was found
    if matches and matches.groupdict():
        group_dict = matches.groupdict()
        date = group_dict.get('date')
        if date:
            # Remove whitespace
            date = date.replace(' ', '')
            # We prefix '20' to the year to make 01/01/17 -> 01/01/2017
            date = date[:6] + '20' + date[6:]
    
    # We identify parsing errors with the 'parse_error'
    return date if date else 'parse_error'

We apply the function to every row and create a new column: 'ad_creation_date_parsed'

In [13]:
ads_df['ad_creation_date_parsed'] = ads_df.ad_creation_date.apply(extract_date_from_string)

We check how many dates could not be parsed:

In [14]:
(ads_df['ad_creation_date_parsed'] == 'parse_error').sum()

1

Since only one date could not be parsed we validate its value:

In [15]:
row = ads_df[ads_df['ad_creation_date_parsed'] == 'parse_error']
row

,file_name,ad_targeting_interests,ad_targeting_people_who_match,ad_impressions,ad_clicks,ad_spend,ad_creation_date,ad_end_date,ad_creation_date_parsed
1340,P(1)0005330.txt,"Veterans, United States Department of Veterans...",NaN,15,2,50.06 RUB,02/21117 01:13:46 AM PST,NaN,parse_error


In this case the date should be 02/21/2017. An l was mistaken to a 1. We replace it manually.

In [16]:
ads_df.loc[row.index, 'ad_creation_date_parsed'] = '02/21/2017'
ads_df.loc[row.index]

,file_name,ad_targeting_interests,ad_targeting_people_who_match,ad_impressions,ad_clicks,ad_spend,ad_creation_date,ad_end_date,ad_creation_date_parsed
1340,P(1)0005330.txt,"Veterans, United States Department of Veterans...",NaN,15,2,50.06 RUB,02/21117 01:13:46 AM PST,NaN,02/21/2017


Now that all dates have been parsed, we replace the original column with the parsed one and remove the temporary parsed column. Since no columns were lost in the process we will not be adding an entry to the summary.

In [17]:
# Replace original column with parsed
ads_df['ad_creation_date'] = ads_df['ad_creation_date_parsed']

# Drop temporary parsed column
ads_df = ads_df.drop(columns=['ad_creation_date_parsed'])

We now execute the same steps for the end date.

In [18]:
ads_df['ad_end_date_parsed'] = ads_df.ad_end_date.apply(extract_date_from_string)

We check how many dates could not be parsed:

In [19]:
(ads_df['ad_end_date_parsed'] == 'parse_error').sum()

0

In [20]:
ads_df['ad_end_date'] = ads_df['ad_end_date_parsed']
ads_df = ads_df.drop(columns=['ad_end_date_parsed'])

Now that both ad_start_date and ad_end_date are properly parsed strings, we can apply a pandas function to transform them into datetime objects. This will make date handling easier during our analysis.

In [21]:
ads_df.ad_creation_date = ads_df.ad_creation_date.apply(lambda date_string : pd.to_datetime(date_string, format='%m/%d/%Y'))
ads_df.ad_end_date = ads_df.ad_end_date.apply(lambda date_string : pd.to_datetime(date_string, format='%m/%d/%Y'))

# Output first 3 rows
ads_df.head(3)

,file_name,ad_targeting_interests,ad_targeting_people_who_match,ad_impressions,ad_clicks,ad_spend,ad_creation_date,ad_end_date
1,P(1)0002823.txt,NaN,"Interests: Pan-Africanism, African-American Ci...",10496,1823,200.00 RUB,2017-04-21,2017-04-22
2,P(1)0002837.txt,NaN,"Interests: Pan-Africanism, African-American Ci...",16305,1337,499.49 RUB,2017-04-13,2017-04-14
5,P(1)0006304.txt,NaN,"Interests: Martin Luther King, Jr., Stop Racis...",8210,1788,"1,570.03 RUB",2017-05-29,2017-05-29


### Parsing ad_spend

Sometimes the ad_spend field contains spaces, dots instead of commas to seperate thousands and the 'RUB' currency shorthand. We use a regular expression to extract the amount of the ad_spend field. We then convert the string to a float.

In [22]:
ads_df['ad_spend']

1          200.00 RUB
2          499.49 RUB
5        1,570.03 RUB
6       3, 106.28 RUB
7          600.00 RUB
            ...      
3511       199.96 RUB
3512       500.00 RUB
3513       401.61 RUB
3515       300.00 RUB
3516       500.00 RUB
Name: ad_spend, Length: 2450, dtype: object

In [23]:
# Pre compile regex for performance
amount_regex = re.compile(r'(?P<amount>([0-9]{1,3}(\.|,)?)+(\.|,)?[0-9]{2})')

# Function returns 'parse_error' on failure to parse and null if the
# input string was null or the string 'None'
def extract_amount_from_string(string):
    matches = None
    amount = None
    
    # If the string is not null or 'None' search for matches
    if not pd.isnull(string) and string != 'None':
        matches = amount_regex.search(string)
    else:
        # null value for string in pandas
        amount = np.nan

    # If the amount was found
    if matches and matches.groupdict():
        group_dict = matches.groupdict()
        amount = group_dict.get('amount')
        if amount:
            # Remove whitespace
            amount = amount.replace(' ', '')
            
            # Remove dots and commas
            amount = amount.replace('.', '').replace(',', '')
            
            # Add a dot two digits form the end
            amount = amount[:-2] + '.' + amount[-2:]
    
    # Return a parse_error if amount parsing failed
    return amount if amount else 'parse_error'

We run the function over our dataset and output the number of parsing errors we've encountered.

In [24]:
ads_df['ad_spend_parsed'] = ads_df.ad_spend.apply(extract_amount_from_string)
(ads_df['ad_spend_parsed'] == 'parse_error').sum()

0

We validate nan values and remove them from the dataset.

In [25]:
cleaning_summary_format['none_ad_spend_count'] = (~pd.isnull(ads_df['ad_spend_parsed'])).sum()
print('There are a total of ' + str(pd.isnull(ads_df['ad_spend_parsed']).sum()) + ' nan values.')

There are a total of 8 nan values.


In [26]:
ads_df[pd.isnull(ads_df['ad_spend_parsed'])]

,file_name,ad_targeting_interests,ad_targeting_people_who_match,ad_impressions,ad_clicks,ad_spend,ad_creation_date,ad_end_date,ad_spend_parsed
174,P(1)0006107.txt,NaN,Interests: Music or Rock music And Must Also M...,281,1,None,2016-05-10,NaT,NaN
604,P(1)0006115.txt,NaN,Interests: Music or Rock music And Must Also M...,327,1,None,2016-05-10,NaT,NaN
934,P(1)0006011.txt,NaN,Behaviors: Facebook access (browser): Chrome A...,404,1,None,2016-05-12,NaT,NaN
1137,P(1)0003195.txt,NaN,Interests: Immigration or Conservatism,100,2,None,2016-01-14,NaT,NaN
1887,P(1)0003175.txt,NaN,NaN,277,5,None,2016-03-10,NaT,NaN
2772,P(1)0005995.txt,NaN,Behaviors: Facebook access (browser): Chrome A...,277,1,None,2016-05-12,NaT,NaN
3013,P(1)0006045.txt,NaN,Behaviors: Facebook access (browser): Chrome A...,251,1,None,2016-05-12,NaT,NaN
3313,P(1)0001473.txt,NaN,"Interests: BlackNews.com, Black (Color) or Huf...",52,1,None,2016-01-14,NaT,NaN


In [27]:
# Remove nulls
ads_df = ads_df[~pd.isnull(ads_df['ad_spend_parsed'])]

# Replace ad_spend with the parse column
ads_df['ad_spend'] = ads_df['ad_spend_parsed']

# Drop the parsed column
ads_df = ads_df.drop(columns=['ad_spend_parsed'])

We transform the ad_spend field from string into a float.

In [28]:
ads_df['ad_spend'] = ads_df['ad_spend'].astype(float)

We validate that all values are positive and remove other values after validation.

In [29]:
print('There are ' + str((ads_df['ad_spend'] > 0).sum()) + ' positive values and a total of ' + str(len(ads_df)) + ' entries.')

There are 2440 positive values and a total of 2442 entries.


In [30]:
cleaning_summary_format['non_positive_ad_spend_count'] = (ads_df['ad_spend'] > 0).sum()
ads_df[ads_df['ad_spend'] <= 0]

,file_name,ad_targeting_interests,ad_targeting_people_who_match,ad_impressions,ad_clicks,ad_spend,ad_creation_date,ad_end_date
843,P(1)0001447.txt,NaN,"Interests: Black Tea Patriots, Black Knowledge...",39,1,0.0,2016-01-14,NaT
2109,P(1)0001588.txt,NaN,Interests: National Museum of American History...,47,1,0.0,2016-01-14,NaT


We remove the two entries with values equal to zero and print out the summary.

In [31]:
ads_df = ads_df[ads_df['ad_spend'] > 0]

cleaning_summary_format['before_ad_spend_count'] = cleaning_summary_format['zeros_ad_clicks_count']

# Reporting
print('''Before formating ad_spend our dataset had {before_ad_spend_count} columns.
After removing rows with 'None': {none_ad_spend_count} columns.
After removing rows with 0 ad clicks: {non_positive_ad_spend_count} columns.'''.format(**cleaning_summary_format))

Before formating ad_spend our dataset had 2450 columns.
After removing rows with 'None': 2442 columns.
After removing rows with 0 ad clicks: 2440 columns.


### Parsing ad_targeting_interests & ad_targeting_people_who_match

The ad_targeting_interests column is split between its own column and a portion of the ad_targeting_people_who_match column's string. To make treatment of this column simpler, our first step will be to extract the 'interest' portion of ad_targeting_people_who_match. We will then parse the ad_targeting_interests column and combine the result.

First we take a look at ad_targeting_people_who_match for entries with and without 'Interests'. We will investigate those without 'Interests' first.

In [32]:
count_null = 0
count_interests = 0
count_other = 0
for s in ads_df['ad_targeting_people_who_match']:
    if pd.isnull(s):
        count_null += 1
    elif 'Interests' in s:
        count_interests += 1
    else:
        count_other +=1
        print(s)

People who like Don't Shoot, Friends of connections: Friends of people who are connected to Don't Shoot
People who like Being Patriotic, Friends of connections: Friends of people who are connected to Being Patriotic
People who like United Muslims of America, Friends of connections: Friends of people who are connected to United Muslims of America
People who like Black Matters, Friends of connections: Friends of people who are connected to Black Matters
People who like Black Matters. Friends of connections: Friends of people who are connected to Black Matters
Politics: Likely to engage with political content (liberal)
People who like LGBT United, Friends of connections: Friends of people who are connected to LGBT United
People who like Black Matters, Friends of connections: Friends of people who are connected to Black Matters
People who like LGBT United, Friends of connections: Friends of people who are connected to LGBT United
Behaviors: African American (US)
People who like LGBT United

In [33]:
print('Null:' + str(count_null) +
      ' Interests: ' + str(count_interests) +
      ' Other: ' + str(count_other) +
      ' Total: ' + str(count_null + count_interests + count_other))

Null:822 Interests: 1243 Other: 375 Total: 2440


From this print out, we see that although some rows belonging to the "Other" category have the 'interests' field missing, we can grab a proxy by taking the "like" groups. We can grab the value of the "like" groups correctly by taking the string after 'Friends of people who are connected to'. We've created the function treat_string_with_friends below to treat these entries.

To treat the rows with 'Interests', we create the funciton treat_string_with_interest. After looking at a few of the raw_files, we see that ad_targeting_people_who_match sometimes contains other fields. To identify rows which had additonal fields, we looked for the number of ':' characters, we then identified patterns in those strings that didn't match the interests field. These patterns are used in the treat_string_with_interest function below.

The crop_to_interest function was created to dynamically use the method when interests are present or not.

In [34]:
# Utility function to crop everything after a given word
def crop_everything_after(string, contains):
    return string[:string.index(contains)] if contains in string else string

# Returns a string containing everything after 'Friends of people who are connected to '
def treat_string_with_friends(string):
    friends_string = 'Friends of people who are connected to '
    start = string.index(friends_string)
    return string[start+len(friends_string):]

# Treat the 
def treat_string_with_interest(string):
    # Crop everything before 'Interests'
    string = string[string.index('Interests'):]

    # Strings identified by visual inspections of entries
    crop_after = [
        'And Must Also Match',
        'School:',
        'Behaviors:',
        'expansion:',
        'Job title:',
        'Multicultural Affinity:',
        'Politics:',
        'Employers:',
        'Field of study:'
    ]

    for to_crop in crop_after:
        string = crop_everything_after(string, to_crop)

    # Finally this substring had a typo
    if 'Stop Racism!:.' in string:
        string = string.replace('Stop Racism!:.', 'Stop Racism!!,')
    
    return string

# If Interests is part of the string use the interest
# method otherwise use the crop to like method.
def crop_to_interest(string):
    if not pd.isnull(string):
        
        if 'Interests' in string:
            string = treat_string_with_interest(string)
        elif 'Friends of people who are connected to ' in string:
            string = treat_string_with_friends(string)
        else:
            # pd.isnull or does not contain interests nor likes
            string = np.nan

    return string

In [35]:
ads_df['ad_targeting_people_who_match'] = ads_df['ad_targeting_people_who_match'].apply(crop_to_interest)

During this operation we lost a few rows that could not be parsed as it did not contain interests.

In [36]:
cleaning_summary_format['null_people_who_match_count'] = pd.isnull(ads_df['ad_targeting_people_who_match']).sum() - count_null
print(str(cleaning_summary_format['null_people_who_match_count']) + ' rows where lost.')

29 rows where lost.


The last cleaning step for this field is to remove the 'Interests' keyword which is sometimes followed by a colon. We use a regular expression to replace this string.

In [37]:
interests_regex = re.compile(r'Interests\s*:?')

def remove_interests_marker(string):
    if not pd.isnull(string):
        string = interests_regex.sub('', string)
    return string

In [38]:
ads_df['ad_targeting_people_who_match'] = ads_df['ad_targeting_people_who_match'].apply(remove_interests_marker)

In [39]:
ads_df.head(3)

,file_name,ad_targeting_interests,ad_targeting_people_who_match,ad_impressions,ad_clicks,ad_spend,ad_creation_date,ad_end_date
1,P(1)0002823.txt,NaN,"Pan-Africanism, African-American Civil Rights...",10496,1823,200.00,2017-04-21,2017-04-22
2,P(1)0002837.txt,NaN,"Pan-Africanism, African-American Civil Rights...",16305,1337,499.49,2017-04-13,2017-04-14
5,P(1)0006304.txt,NaN,"Martin Luther King, Jr., Stop Racism!!, Afric...",8210,1788,1570.03,2017-05-29,2017-05-29


We now do the same exercise with ad_targeting_interests. We first identify non-null rows that may contain an extra field. We do so by looking for the ':' character and printing out these rows.

In [40]:
non_null_interests = ads_df[~pd.isnull(ads_df['ad_targeting_interests'])]['ad_targeting_interests']

for row_with_colon in non_null_interests[non_null_interests.str.contains(':')]:
    print(row_with_colon)

BlackNews.com or HuffPost Black Voices Behaviors: African American (US)
BlackNews.com or HuffPost Black Voices Behaviors: African American (US)
Humanitarianism, Human rights or Humanitarian aid Behaviors: African American (US)
Black Power Behaviors: Multicultural Affinity: African American (US)
Human rights or Malcolm X Behaviors: African American (US)
BlackNews.com or HuffPost Black Voices Behaviors: African American (US)
BlackNews.com or HuffPost Black Voices Behaviors: African American (US)
BlackNews.com or HuffPost Black Voices Behaviors: African American (US)
BlackNews.com or HuffPost Black Voices Behaviors: African American (US)
BlackNews.com or HuffPost Black Voices Behaviors: African American (US)
BlackNews.com or HuffPost Black Voices Behaviors: African American (US)
Humanitarianism, Human rights or Humanitarian aid Behaviors: African American (US)
Muslims Are Not Terrorists. Islamism or Muslim Brotherhood Connections: People who like United Muslims of America
History Politics

Most of these rows seem to contain an additional field "Behaviors" we will remove it from the rows as we can easily use the interest part to identify the targeted demographic.

In [41]:
# Removes additional section part of the ad_targeting_interests string
def treat_interest(string):
    if not pd.isnull(string):
        # Strings identified by visual inspections of entries
        crop_after = [
            'And Must Also Match',
            'School:',
            'Behaviors:',
            'expansion:',
            'Job title:',
            'Multicultural Affinity:',
            'Politics:',
            'Employers:',
            'Field of study:',
            'Connections:',
            'Home Composition:'
        ]

        for to_crop in crop_after:
            string = crop_everything_after(string, to_crop)
    else:
        # pd.isnull value for strings
        string = np.nan
    
    return string

ads_df['ad_targeting_interests'] = ads_df['ad_targeting_interests'].apply(treat_interest)

In [42]:
non_null_interests = ads_df[~pd.isnull(ads_df['ad_targeting_interests'])]['ad_targeting_interests']
print('After treatment, there are ' + str(non_null_interests[non_null_interests.str.contains(':')].count()) +' rows with more than one field.')

After treatment, there are 0 rows with more than one field.


Now that both ad_targeting_interests and ad_targeting_people_who_match have been cleaned, we can now merge the two columns into one. First let's verify that there are no rows where both columns are non-null or both null.

In [44]:
def interests_both_null(row):
    return (pd.isnull(row.ad_targeting_interests) and pd.isnull(row.ad_targeting_people_who_match))

def interests_both_non_null(row):
    return (not pd.isnull(row.ad_targeting_interests) and not pd.isnull(row.ad_targeting_people_who_match))
    
# How many rows have both columns as null      
cleaning_summary_format['null_all_interests_columns_count'] = ads_df.apply(interests_both_null, axis=1).sum()


# How many rows have both columns populated
non_null_count = ads_df.apply(interests_both_non_null, axis=1).sum()

print('We have a total of ' + str(both_null) + ' rows with both columns null and a total of ' + str(non_null_count) + ' rows which have both values set.')

We have a total of 214 rows with both columns null and a total of 0 rows which have both values set.


We drop rows that do not contain interests information in both columns. We will merge the other rows by replacing the values of ad_targeting_interests with ad_targeting_people_who_match.

In [45]:
def merge_interests(row):
    return row.ad_targeting_interests if not pd.isnull(row.ad_targeting_interests) else row.ad_targeting_people_who_match

# Merge interests
ads_df['ad_targeting_interests'] = ads_df.apply(merge_interests, axis=1)

# Drop 'ad_targeting_people_who_match'
ads_df = ads_df.drop(columns=['ad_targeting_people_who_match'])

# Drop null columns
ads_df = ads_df[(~pd.isnull(ads_df['ad_targeting_interests']))]

### Writing to file

In [192]:
ads_df.head(3)

,file_name,ad_targeting_interests,ad_impressions,ad_clicks,ad_spend,ad_creation_date,ad_end_date
1,P(1)0002823.txt,"Pan-Africanism, African-American Civil Rights...",10496,1823,200.00,2017-04-21,2017-04-22
2,P(1)0002837.txt,"Pan-Africanism, African-American Civil Rights...",16305,1337,499.49,2017-04-13,2017-04-14
5,P(1)0006304.txt,"Martin Luther King, Jr., Stop Racism!!, Afric...",8210,1788,1570.03,2017-05-29,2017-05-29


In [193]:
ads_df.to_csv('../clean_data/clean_data.csv', index=None, header=True)

## Data cleaning summary

We have lost rows for various reasons during the cleaning:

| Rows lost | Reason |
|------|------|
| 20 | Null ad_creation_date column|
| 1047 | Missing creation_date|
| 10 | Improper input of ad_spend|
| 29 | Interests missing from people_who_match |
|214 | Interests missing from both people_who_match and interests |

Finally, we have decided to keep the following fields:

| Field name             | Type     | Description                         |
|------------------------|----------|-------------------------------------|
| ad_targeting_interests | string   | Interests used to target users      |
| ad_impressions         | int      | Number of users who saw the ads     |
| ad_clicks              | int      | Number of times the ads was clicked |
| ad_spend               | float    | Money spent on the ad in RUB        |
| ad_creation_date       | datetime | Creation date of the ad             |
| ad_end_date            | datetime | Date at which the ad stopped        |

---

You can now proceed to the [demographic_labeling](demographic_labeling.ipynb) notebook.